In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
import sys
sys.path.insert(0, '..')
from odynn.circuit import Circuit
from odynn.models import LeakyIntegrate, ChemSyn
import numpy as np
import torch

DT = 1
N_parallel = 200
connections = [(0,1) , (1,0)]
ts = np.arange(0.0, 1500., DT)

def get_circ():
    n = LeakyIntegrate.create_default(2, N_parallel, tensors=True, dt=DT)
    p = ChemSyn.get_random(1,N_parallel)
    p = {k: np.concatenate([v,v]) for k,v in p.items()}
    s = ChemSyn([c[0] for c in connections], [c[1] for c in connections], 
                              init_p=p, tensors=True, dt=DT)
    return Circuit(n, s)

In [3]:
import pylab as plt
import seaborn as sns
import pickle

with open('cur', 'rb') as f:
        i_f = pickle.load(f)
def create():
    i_f = np.random.normal(size=(len(ts), 2))
    i_f[:,0] = np.convolve(i_f[:,0], [1/8 for _ in range(20)], 'same')
    i_f[:,1] = np.convolve(i_f[:,1], [1/8 for _ in range(20)], 'same')
    with open('cur2', 'wb') as f:
        pickle.dump(i_f, f)
    return i_f
i_forced = torch.Tensor(i_f)# 3 * torch.randn(i_f.shape)#torch.Tensor(i_f) + 2 * torch.randn(i_f.shape)
print(i_forced.shape)

def plot(tr, c=None):
    plt.figure(figsize=(10,3))
    if c is None:
        plt.plot(tr)
    else:
        plt.plot(tr, c)
    plt.xlabel('time (s)')
    sns.despine()
    plt.show()
    
plot(i_forced.numpy()[:,0])
plot(i_forced.numpy()[:,1], 'darkorange')
plot(i_forced.numpy())

x = np.arange(-80., -40.)
def obj(x):
    return 225 / (85 + x) - 85

plt.plot(x, obj(x))
print(obj(-80), obj(-40))
plt.show()

torch.Size([1500, 2])


<Figure size 720x216 with 1 Axes>

<Figure size 720x216 with 1 Axes>

<Figure size 720x216 with 1 Axes>

-40.0 -80.0


<Figure size 432x288 with 1 Axes>

In [41]:
from tqdm import tqdm
import seaborn as sns

init = np.array([[-40,-80]])
print(init.shape)
init = np.repeat(init[:, :, None], N_parallel, axis=-1)
init = init[:, None]
print(init.shape)

"""Optimize correlation"""

circuit = get_circ()
losses = []
gs = []
# Define loss function
loss_fn = torch.nn.MSELoss(reduction='mean')
learning_rate = 0.1
optimizer = torch.optim.Adam(circuit._synapses.parameters.values(), lr=learning_rate)
# print(circuit._synapses.parameters)

for t in tqdm(range(501)):
    y = circuit.calculate(i_forced, init)
    # Compute MSE
    
#     v0 = y[:,0,0] - torch.mean(y[:,0,0], dim=0)
#     v1 = y[:,0,1] - torch.mean(y[:,0,1], dim=0)

#     cost = torch.sum(v0 * v1, dim=0) / (torch.sqrt(torch.sum(v0 ** 2, dim=0)) * torch.sqrt(torch.sum(v1 ** 2, dim=0)))
#     loss = (cost - torch.Tensor([-1])).pow(2)

    loss = (obj(y[:,0,0]) - y[:,0,1]).pow(2).mean(0)
    
    # Equal max
    loss += (y[:,0,0].max(0)[0] - y[:,0,1].max(0)[0]).pow(2)
    loss += (y[:,0,0].min(0)[0] - y[:,0,1].min(0)[0]).pow(2)
#     loss += (y[10:,0,0].min(0)[0] + 80).pow(2)
#     loss += (y[10:,0,1].max(0)[0] + 40).pow(2)
#     loss += (y[10:,0,1].min(0)[0] + 80).pow(2)
    
    losses.append(loss)
    
    gs.append(circuit._synapses.parameters['G'][0].detach().numpy())
    #Upgrade variables
    optimizer.zero_grad()
    loss.mean().backward()
    with torch.no_grad():
        for p in circuit._synapses.parameters.values():
            p.grad = torch.stack([p.grad.sum(0), p.grad.sum(0)])
    optimizer.step()

    circuit.apply_constraints()
    
    if t%25 == 0:
        print(loss.mean().detach().numpy(), loss.min().detach().numpy())
        best = loss.argmin()
        print({k: v[0,best] for k,v in circuit._synapses.parameters.items()})
#         for n in range(N_parallel):
        plt.subplot(211)
        plt.figure(figsize=(10,5))
        plt.plot(y[:,0,:,best].detach().cpu().numpy())
        plt.title('Voltage (mV)')
        plt.subplot(212)
        plt.plot(i_forced.numpy())
        plt.plot(y[:,0,0,best].detach().cpu().numpy(), obj(y[:,0,0,best].detach().cpu().numpy()), 'r')
        plt.plot(y[:,0,0,best].detach().cpu().numpy(), y[:,0,1,best].detach().cpu().numpy())
        sns.despine(bottom=True)
        plt.xticks([])
        plt.show()
        data0={k: v.detach().numpy()[0] for k,v in circuit._synapses.parameters.items()}
        plt.scatter(data0['G'], data0['mdp'], c=data0['E'])
        plt.colorbar()
#         sns.scatterplot(x='G', y='mdp', hue='E', data={k: v.detach().numpy()[0] for k,v in circuit._synapses.parameters.items()})
#         sns.scatterplot(x='G', y='mdp', hue='E', data={k: v.detach().numpy()[1] for k,v in circuit._synapses.parameters.items()})
#         print(np.corrcoef(i_forced[:,0].detach().cpu().numpy(), y[:,0].detach().cpu().numpy()) )
#         plt.plot(i.detach().numpy())
        plt.xscale('log')
        plt.show()
        plt.close()

plt.plot([l.detach().numpy() for l in losses], linewidth=0.2)
plt.yscale('log')
plt.show()

  0%|          | 0/501 [00:00<?, ?it/s]

(1, 2)
(1, 1, 2, 200)
635.3706 406.08115
{'G': tensor(1.7343, grad_fn=<SelectBackward>), 'mdp': tensor(-40.6437, grad_fn=<SelectBackward>), 'scale': tensor(9.0189, grad_fn=<SelectBackward>), 'E': tensor(-80.9934, grad_fn=<SelectBackward>)}


<Figure size 720x360 with 1 Axes>

<Figure size 432x288 with 2 Axes>

  5%|▍         | 25/501 [00:20<06:16,  1.26it/s]

...

348.17114 0.2846076
{'G': tensor(1.9839, grad_fn=<SelectBackward>), 'mdp': tensor(-37.6267, grad_fn=<SelectBackward>), 'scale': tensor(0.7581, grad_fn=<SelectBackward>), 'E': tensor(-96.7735, grad_fn=<SelectBackward>)}


<Figure size 720x360 with 1 Axes>

<Figure size 432x288 with 2 Axes>

100%|██████████| 501/501 [08:12<00:00,  1.17s/it]


<Figure size 432x288 with 1 Axes>

In [20]:
# print(best)
# for n in range(N_parallel):
#     print(n)
#     plt.subplot(211)
#     plt.plot(y[:,0,:,n].detach().cpu().numpy())
#     plt.subplot(212)
#     plt.plot(i_forced.numpy())
#     plt.show()
i_f = np.random.normal(size=(len(ts)*10, 2))
i_f[:,0] = np.convolve(i_f[:,0], [1 for _ in range(30)], 'same')
i_f[:,1] = np.convolve(i_f[:,1], [1 for _ in range(30)], 'same')
plt.plot(i_f)

tensor([-0.0748], grad_fn=<DivBackward0>) tensor(14.6043, grad_fn=<MeanBackward1>) tensor(14.6043, grad_fn=<MinBackward1>)


<Figure size 432x288 with 1 Axes>

In [39]:
g = 1.7511355
m = -42.919064
E = -86.07687
s = 0.05

g = 3
# ps = {k: v[:,28].detach().numpy() for k,v in circuit._synapses.parameters.items()}
ps = {'G': np.array([g,g]), 'mdp': np.array([m,m]), 'scale': np.array([0.05, 0.05]), 'E': np.array([E])}
# print(ps)
ps = {k: v[:,None] for k,v in ps.items()}
n = LeakyIntegrate.create_default(2, 1, dt=0.1)
print(n._param)
s = ChemSyn([c[0] for c in connections], [c[1] for c in connections], 
                              init_p=ps, dt=DT)
circ = Circuit(n, s)

# plt.plot(i_f)
# plt.show()
plt.figure(figsize=(14,6))
X = circ.calculate(i_f)

plt.plot(X[:,0,0,:,0])
sns.despine(bottom=True)
# plt.axhline(-40.8, linestyle='--', color='r', label = 'resting potential')
# plt.axhline(-42.92, linestyle='--', color='k', label = 'midpoint')
plt.legend()
plt.xticks([])
plt.title('Voltage (mV)')
plt.show()
plt.figure(figsize=(14,3))

l=0.8
plt.plot(i_f[:,0], linewidth=l)
plt.plot(i_f[:,1], linewidth=l)
sns.despine()
plt.show()


{'C_m': array([[5.],
       [5.]]), 'g_L': array([[0.272],
       [0.272]]), 'E_L': array([[-40.8],
       [-40.8]])}


No handles with labels found to put in legend.


<Figure size 1008x432 with 1 Axes>

<Figure size 1008x216 with 1 Axes>